# DS-SF-34 | Unit Project | 3 | Machine Learning Modeling and Executive Summary | Starter Code

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.  You will summarize and present your findings and the methods you used.

In [1]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

import statsmodels.formula.api as smf

from sklearn import linear_model

In [33]:
df = pd.read_csv(os.path.join('..', '..', 'dataset', 'dataset-ucla-admissions.csv'))
df.dropna(inplace = True)

df

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
...,...,...,...,...
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0


## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether an applicant was admitted.

In [13]:
# TODO
df.prestige.value_counts()

2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64

In [18]:
pd.crosstab(df.prestige, 'admit')

col_0,admit
prestige,
1.0,61
2.0,148
3.0,121
4.0,67


## Part B.  Feature Engineering

> ### Question 2.  Create a one-hot encoding for `prestige`.

In [34]:
# TODO
presdf = pd.get_dummies(df.prestige)
presdf

,1.0,2.0,3.0,4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
395,0,1,0,0
396,0,0,1,0
397,0,1,0,0
398,0,1,0,0


> ### Question 3.  How many of these binary variables do we need for modeling?

Answer: 3! The model should assume 3 0's means the last variable

> ### Question 4.  Why are we doing this?

Answer: To be able to model the classification variable, prestige

> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [35]:
# TODO
df = df.join([presdf])
df

,admit,gre,gpa,prestige,1.0,2.0,3.0,4.0
0,0,380.0,3.61,3.0,0,0,1,0
1,1,660.0,3.67,3.0,0,0,1,0
2,1,800.0,4.00,1.0,1,0,0,0
3,1,640.0,3.19,4.0,0,0,0,1
4,0,520.0,2.93,4.0,0,0,0,1
...,...,...,...,...,...,...,...,...
395,0,620.0,4.00,2.0,0,1,0,0
396,0,560.0,3.04,3.0,0,0,1,0
397,0,460.0,2.63,2.0,0,1,0,0
398,0,700.0,3.65,2.0,0,1,0,0


In [36]:
df = df.drop('prestige', axis = 1)
df

,admit,gre,gpa,1.0,2.0,3.0,4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1
...,...,...,...,...,...,...,...
395,0,620.0,4.00,0,1,0,0
396,0,560.0,3.04,0,0,1,0
397,0,460.0,2.63,0,1,0,0
398,0,700.0,3.65,0,1,0,0


In [39]:
df.columns = ['admit', 'gre', 'gpa', 'one', 'two', 'three', 'four']


In [40]:
df

,admit,gre,gpa,one,two,three,four
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1
...,...,...,...,...,...,...,...
395,0,620.0,4.00,0,1,0,0
396,0,560.0,3.04,0,0,1,0
397,0,460.0,2.63,0,1,0,0
398,0,700.0,3.65,0,1,0,0


## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether an applicant was admitted.

In [41]:
# TODO
pd.crosstab(df.one, 'admit')

col_0,admit
one,
0,336
1,61


> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [46]:
# TODO
p = 61. / (336 + 61) 
print p

0.153652392947


In [52]:
pp = p /(1 - p)
pp

0.18154761904761907

> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [49]:
# TODO
q = 336. / (397)
q

0.8463476070528967

In [51]:
qq = q / (1 - q)
qq

5.5081967213114735

> ### Question 9.  Finally, what's the odds ratio?

In [53]:
# TODO
pp / qq

0.032959537981859424

> ### Question 10.  Write this finding in a sentence.

Answer: The odds of admission when from a prestige 1 school is significantly lower than from other schools.

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentence.

In [54]:
# TODO
pd.crosstab(df.four, 'admit')

col_0,admit
four,
0,330
1,67


In [104]:
p = 67. / (330 + 67)
q = 330. / (397)
pp = p / (1-p)
qq = q / (1-q)
pp / qq

0.04122130394857666

Answer: Being from a prestige 4 lowers your chances of entry into UCLA

## Part D. Analysis using `statsmodels`

> ### Question 12.  Fit a logistic regression model predicting admission into UCLA using `gre`, `gpa`, and the `prestige` of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [57]:
# TODO
admit = df.admit
X = df[['gre', 'gpa', 'two', 'three', 'four']]
print admit
X

0      0
1      1
2      1
3      1
4      0
      ..
395    0
396    0
397    0
398    0
399    0
Name: admit, dtype: int64


,gre,gpa,two,three,four
0,380.0,3.61,0,1,0
1,660.0,3.67,0,1,0
2,800.0,4.00,0,0,0
3,640.0,3.19,0,0,1
4,520.0,2.93,0,0,1
...,...,...,...,...,...
395,620.0,4.00,1,0,0
396,560.0,3.04,0,1,0
397,460.0,2.63,1,0,0
398,700.0,3.65,1,0,0


In [68]:
smf.Logit(admit, X).fit()

Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


> ### Question 13.  Print the model's summary results.

In [66]:
# TODO
smf.Logit(admit, X).fit().summary()


Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Sun, 11 Jun 2017   Pseudo R-squ.:                 0.05722
Time:                        09:58:59   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0014      0.001      1.308      0.191        -0.001     0.003
gpa           -0.1323      0.195     -0.680      0.497        -0.514     0.249
two           -0.9562      0.302     -3.171      0.002        -1.547    -0.365
three         -1.5375      0.332     -4.627      0.000        -2.189    -0.886
four          -1.8699      0.401     -4.658      0.000        -2.657    -1.083
==============================================================================
"""

In [72]:
X['intercept'] = 1.0
smf.Logit(admit, X).fit().summary()
# Now everything is significant

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


/Users/peterr/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 11 Jun 2017   Pseudo R-squ.:                 0.08166
Time:                        10:19:13   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
two           -0.6801      0.317     -2.146      0.032        -1.301    -0.059
three         -1.3387      0.345     -3.882      0.000        -2.015    -0.663
four          -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
==============================================================================
"""

> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

In [73]:
smf.Logit(admit, X).fit().conf_int()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


,0,1
gre,0.000074,0.004362
gpa,0.127619,1.431056
two,-1.301337,-0.058936
three,-2.014579,-0.662776
four,-2.371624,-0.735197
intercept,-6.116077,-1.637631


In [74]:
# TODO
print np.exp(smf.Logit(admit, X).fit().params)

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6
gre          1.002221
gpa          2.180027
two          0.506548
three        0.262192
four         0.211525
intercept    0.020716
dtype: float64


> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

Answer: Compared to going to a prestige 1 school, a prestige 2 school is half as likely to get in

> ### Question 16.  Interpret the odds ratio of `gpa`.

Answer: A unit increase in gpa makes it more than twice as likely to get in compared with the unit below

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [111]:
# TODO
result = smf.Logit(admit, X).fit()
result.predict([[800, 4.0, 0, 0 ,0]])
# ~64% chance of admission

Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


array([ 0.63739858])

In [112]:
result.predict([[800, 4.0, 1, 0 ,0]])
# ~40% chance

array([ 0.40320425])

In [114]:
result.predict([[800, 4.0, 0, 1 ,0]])
# ~ 27% chance

array([ 0.27420161])

In [113]:
result.predict([[800, 4.0, 0, 0 ,1]])
# ~ 21% chance

array([ 0.21318433])

Answer: TODO

## Part E. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [82]:
# TODO
admit = df.admit
X = df[['gre', 'gpa', 'two', 'three', 'four']]
model_admit = linear_model.LogisticRegression(C = 10 **2).fit(X, admit)
print model_admit.coef_
print model_admit.intercept_

[[ 0.00215822  0.67315495 -0.62882239 -1.25222745 -1.56879212]]
[-3.51478687]


> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [83]:
# TODO
print np.exp(model_admit.coef_)

[[ 1.00216055  1.96041259  0.53321936  0.28586733  0.20829663]]


Answer: They are slightly different which could be rounding error or the difference in how things are calculated.

> ### Question 20.  Again, assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [107]:
# TODO
model_admit.predict_proba([[800, 4.0, 0,0, 0]])
# 71% chance of admission

array([[ 0.28814605,  0.71185395]])

In [108]:
model_admit.predict_proba([[800, 4.0, 1,0, 0]])
# 57% chance of admission

array([[ 0.43153702,  0.56846298]])

In [109]:
model_admit.predict_proba([[800, 4.0, 0,1, 0]])
# 41% chance of admission

array([[ 0.58608936,  0.41391064]])

In [110]:
model_admit.predict_proba([[800, 4.0, 0,0, 1]])
# 34% chance of admission

array([[ 0.66024514,  0.33975486]])

Answer: TODO

## Part F.  Executive Summary

> ## Question 21.  Introduction
>
> Write a problem statement for this project.

Answer: Our graduate programs have seen a marked increase in the amount of applications received each year. To better handle the increased volume we need to reassess our standards and look at the prior candidate pools and which attribute contributed to their acceptance into our school. We need to have a new process and standards at least 3 months before the next application deadline.

> ## Question 22.  Dataset
>
> Write up a description of your data and any cleaning that was completed.

Answer: We have 3 variable on which to predict admission. gre is the GRE score. gpa is the student's undergraduate GPA. And prestige is the level of prestige of the alma mater classified into 4 levels.

Variable | Description | Type of Variable
---|---|---
admit | 0 = Not admitted, 1 = Admitted | Categorical
gre | Score on Graduate Record Examination | Discrete
gpa | Score of Grade Point Average | Continuous
prestige | Prestige of alma mater (4 = lowest, 1 = highest) | Categorical

> ## Question 23.  Demo
>
> Provide a table that explains the data by admission status.

Answer: TODO

> ## Question 24.  Methods
>
> Write up the methods used in your analysis.

Answer: Our goal for our problem is to better understand who we admit, so for our analysis we utilized logistic regression. With 'prestige' as our categorical variable we needed to split it up so we could see the effect of each level separately, so we created dummy variables. With dummy variables we were able to run our regression and find coeficients for our variables. With our coeficients and odds ratios we are able to predict whether we will admit a student.

> ## Question 25.  Results
>
> Write up your results.

Answer: After running the regression using two different tools we found that the probability of a prestige 1 student being admitted when GRE is 800 and GPA is 4.0 is about 14% higher than a prestige 2. Prestige 2 has a 13-16% chance over prestige 3. Prestige 3 has a 6-7% higher chance over prestige 4. A prestige 4 student has roughly a 21-34% chance to be accepted with a 800 GRE and 4.0 GPA.

> ## Question 26.  Visuals
>
> Provide a table or visualization of these results.

Answer: TODO

> ## Question 27.  Discussion
>
> Write up your discussion and future steps.

Answer: There is a significant bias towards students of higher prestige when all other factors are kept the same. When two students have the same academic achievement, but when alma mater increases chance of acceptance by over 100% there is a clear problem. Moving forward we need to balance who we're accepting or keep our inherit biases in mind when making decisions and rerun our analysis year over year to see if we are improving.